# Présentation du projet : 

## Contexte 

Vous êtes consultant pour Olist, une solution de vente sur les marketplaces en ligne.

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Votre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

Vous devrez fournir à l’équipe marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.


## Données 

Pour cette mission, Olist vous fournit une base de données anonymisée comportant des informations sur l’historique de commandes, les produits achetés, les commentaires de satisfaction, et la localisation des clients depuis janvier 2017.

## Mission

Votre mission est d’aider les équipes d’Olist à comprendre les différents types d'utilisateurs. Vous utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires. Ces catégories pourront être utilisées par l’équipe marketing pour mieux communiquer.

Pour des raisons de confidentialité, Olist ne fournit pas beaucoup de données, à vous de fouiller dans celles dont vous disposez et de créer les meilleures features pour les exploiter.

Enfin, votre client, Olist, a spécifié sa demande ainsi :

*La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.*


# Ressources : 

## Les données et leur description :

Site d'Olist (en portugais) : https://olist.com/

Adresse des données : https://www.kaggle.com/olistbr/brazilian-ecommerce

Convention PEP8 : https://www.python.org/dev/peps/pep-0008/


In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data_path = "/home/clairegayral/Documents/openclassroom/data/P4/"
res_path = "/home/clairegayral/Documents/openclassroom/res/P4/"

## my .py : 
from script00_lists_of_vars import *
from script01_duplicates import *
from script02_missing_values_treatment import *
from script03_univariate_analysis import *
from script04_multivariate_analysis import *

from sklearn import model_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.impute import KNNImputer
from sklearn import linear_model


In [66]:
product_category_name_translation = pd.read_csv(data_path + "product_category_name_translation.csv")
sellers = pd.read_csv(data_path+"olist_sellers_dataset.csv")
products = pd.read_csv(data_path+"olist_products_dataset.csv")
orders = pd.read_csv(data_path+"olist_orders_dataset.csv")
order_reviews = pd.read_csv(data_path+"olist_order_reviews_dataset.csv")
order_payments = pd.read_csv(data_path+"olist_order_payments_dataset.csv")
order_items = pd.read_csv(data_path+"olist_order_items_dataset.csv")
geolocation = pd.read_csv(data_path+"olist_geolocation_dataset.csv")
customers = pd.read_csv(data_path+"olist_customers_dataset.csv")


Je commence par regarder les liens entre les tables. Ce qui nous intéresse est une classification des clients : commençons donc par explorer cette table seule. 

In [67]:
print(customers.shape)
print(customers.info())
customers.head(2)

(99441, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


Petite vérification : Est-ce que les "customer_unique_id" ont bien tous une adresse unique ? 

In [155]:
unique_id = customers["customer_unique_id"]
customer_id_non_unique = unique_id.value_counts()>1
customer_id_non_unique = unique_id.value_counts()[customer_id_non_unique].index
customer_id_non_unique = unique_id.isin(customer_id_non_unique)
customers_non_unique_commande = customers[customer_id_non_unique]
# customers_non_unique_commande.sort_values(by="customer_unique_id")

unique_id = customers_non_unique_commande.iloc[:,1:].drop_duplicates()
unique_id = customers_non_unique_commande.loc[unique_id.index]
unique_id = unique_id["customer_unique_id"]

tmp = unique_id.value_counts()>1
tmp = unique_id.value_counts()[tmp].index
tmp = unique_id.isin(tmp)
tmp = unique_id[tmp].index
customers_non_unique_commande.loc[tmp].sort_values(by="customer_unique_id")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
87012,d95f60d70d9ea9a7fe37c53c931940bb,004b45ec5c64187465168251cd1c9c2f,57035,maceio,AL
72451,49cf243e0d353cd418ca77868e24a670,004b45ec5c64187465168251cd1c9c2f,57055,maceio,AL
61403,f530197ea86ced9488a03d055e118ebf,0058f300f57d7b93c477a131a59b36c3,40731,salvador,BA
36269,8ac44e9c15d396b8c3c7cbab0fff4536,0058f300f57d7b93c477a131a59b36c3,41370,salvador,BA
54038,e024c1515b1062b1e7e613f840c35ba3,012452d40dafae4df401bced74cdb490,3984,sao paulo,SP
...,...,...,...,...,...
84770,e09ccd04cb9679cb5033871d086afd62,fd09c64a101e3eff4adbca1b28552514,8070,sao paulo,SP
76055,15baa4d1c9fd174d26e105980d24aeb9,fe3e52de024b82706717c38c8e183084,72306,brasilia,DF
59713,1e79e4f1a4653e041c3ce1ddb2ff239f,fe3e52de024b82706717c38c8e183084,36420,ouro branco,MG
78212,185d4f6e56ea6a1e423468b4d2eaa7cc,fe59d5878cd80080edbd29b5a0a4e1cf,71065,brasilia,DF


In [102]:
customers_non_unique_commande.loc[:,"customer_city"].value_counts()


sao paulo            1084
rio de janeiro        510
belo horizonte        198
brasilia              126
curitiba              104
                     ... 
salinas                 1
juazeiro do norte       1
canarana                1
viana                   1
aracati                 1
Name: customer_city, Length: 819, dtype: int64

In [ ]:
customers_non_duplicates = customers.drop_duplicates(subset="customer_unique_id")

# y = customers_non_unique_commande["customer_city"]
# customers_non_unique_commande.loc[:,"customer_city"].value_counts()

In [ ]:
y = 
sns.barplot(x=y.index, y=y.values, order =y.sort_values(ascending=False).index)


Il y a donc au total {{customers.shape[0]}} commandes, passées par  {{len(customers["customer_unique_id"].value_counts())}} clients différents. Cette table donne également 3 informations sur la localisation du client (zip code, city, state), contenues dans des variables catégorielles.

A voir s'il ne sera pas pertinent de faire appel à la table ```geolocation``` pour avoir les coordonnées des clients (latitude et longitude). 

In [45]:
geolocation.head(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


Ensuite, nous cherchons à classifier les clients selon leur consommation : regardons donc l'échelle "commande"

In [13]:
# sellers, products, orders, order_reviews, 
# order_items, geolocation, customers

link_customer_order = {}

for customer_id in customers["customer_id"][:15] :
    print(orders.loc[orders["customer_id"]== customer_id, "order_id"])

88492    00e7ee1b050b8499577073aeb2a297a1
Name: order_id, dtype: object
90418    29150127e6685892b6eab3eec79f59c7
Name: order_id, dtype: object
22557    b2059ed67ce144a36e2aa97d2c9e9ad2
Name: order_id, dtype: object
32180    951670f92359f4fe4a63112aa7306eba
Name: order_id, dtype: object
69902    6b7d50bd145f6fc7f33cebabd7e49d0f
Name: order_id, dtype: object
69705    5741ea1f91b5fbab2bd2dc653a5b5099
Name: order_id, dtype: object
14908    36e694cf4cbc2a4803200c35e84abdc4
Name: order_id, dtype: object
68879    1093c8304c7a003280dd34598194913d
Name: order_id, dtype: object
20163    1ebeea841c590e86a14a0d7a48e7d062
Name: order_id, dtype: object
27657    7433cbcc783205509d66a5260da5b574
Name: order_id, dtype: object
48039    8428e578bb1cf839ae26a6b7615502b9
Name: order_id, dtype: object
55343    f86c5ed7048ac10eb88ec21c00f71892
Name: order_id, dtype: object
8261    83d8f70418eefd96f37b483dff430305
Name: order_id, dtype: object
47483    852d2f4d37773bcbc21c8e09a05a4ea5
Name: order_id, dtype: 